In [1]:
# using HERE Javascript Map API
# draw location points as markers and links as lines

epsilon = 1e-8

class Point(object):

    def __init__(self, context):
        self.lat = None
        self.lon = None
        self.color = "orange"
        self.text = ""
        n = len(context)
        if n > 0: self.lat = float(context[0])
        if n > 1: self.lon = float(context[1])
        if n > 2 and context[2] is not None: self.color = context[2]
        if n > 3 and context[3] is not None: self.text = context[3]
        self.slat = str(self.lat)
        self.slon = str(self.lon)

    def interpolate(self, p, frac):
        dlat = p.lat - self.lat
        dlon = p.lon - self.lon
        lat = self.lat + frac * dlat
        lon = self.lon + frac * dlon
        return Point(lat, lon, self.color, self.text)

    def __str__(self):
        ostr = "None"
        if self.lat is not None and self.lon is not None:
            ostr = ",".join([str(x) for x in [self.lat, self.lon, self.color, self.text]])
        return ostr

    def __repr_(self): return self.__str__()

    def equalCoord(self, b):
        if abs(self.lat - b.lat) < epsilon and abs(self.lon - b.lon) < epsilon: return True
        return False

    def __eq__(self, o): return self.equalCoord(o)


class Link(object):

    def __init__(self, context):
        self.linkid = None
        self.shapepoints = None
        self.color = "blue"
        self.text = ""
        self.width = "10"
        n = len(context)
        if n > 0: self.linkid = context[0]
        if n > 1: self.shapepoints = [Point(p) for p in context[1]]
        if n > 2 and context[2] is not None: self.color = context[2]
        if n > 3 and context[3] is not None: self.text = context[3]
        if n > 4 and context[4] is not None: self.width = context[4]

    def shapepoints_str(self):
        ostr = "None"
        if self.shapepoints is not None:
            ostr = ",".join([p.slat+","+p.slon for p in self.shapepoints])
        return ostr

    def __str__(self):
        ostr = "None"
        if self.linkid is not None:
            ostr = ",".join([self.linkid, self.shapepoints_str(), self.color, self.text, self.width])
        return ostr

    def __repr_(self): return self.__str__()

    def __eq__(self, o):
        if self.linkid == o.linkid: return True
        return False


def gethtml(raw_points, raw_links, appId, appCode, map_width="1200px", map_height="600px"):

    """
    a point object consists of (lat, lon, color, text)
    Note: color and text can be omitted.
    Expected Inputs:
        raw_points: (point1, point2, ...)
        raw_links: ((linkid,(point1, point2, ...),color,text,width)
        Note: color, text, width for a link object can be omitted
        map_width (optional, default=1200px): width of map as px(pixel) or %(relative ratio to the browser)
        map_height (optional, default=600px): height of map as px(pixel). It has to be in px.
    gethtml function transforms raw_points, raw_links to lists of Point and Link objects
    and return html document string that can be saved to a file
    """

    # transform inputs to Point objects for convenience
    points = [Point(p) for p in raw_points]
    links = [Link(l) for l in raw_links]

    ostr = """
    <!DOCTYPE html>
    <html>
    <head>
      <meta name="viewport" content="initial-scale=1.0, width=device-width" />
      <link rel="stylesheet" type="text/css"
        href="https://js.api.here.com/v3/3.0/mapsjs-ui.css" />
      <script type="text/javascript" charset="UTF-8"
        src="https://js.api.here.com/v3/3.0/mapsjs-core.js"></script>
      <script type="text/javascript" charset="UTF-8"
        src="https://js.api.here.com/v3/3.0/mapsjs-service.js"></script>
      <script type="text/javascript" charset="UTF-8"
        src="https://js.api.here.com/v3/3.0/mapsjs-ui.js"></script>
      <script type="text/javascript" charset="UTF-8"
        src="https://js.api.here.com/v3/3.0/mapsjs-mapevents.js"></script>
    </head>
    """

    ostr += "<body>\n"

    filled = len(points) > 0 or len(links) > 0

    #ostr += '<div id="map" style="width: 100%; height: 85vh" />'
    ostr += '<div id="map" style="width: '+map_width+'; height: '+map_height+'" />'

    ostr += """
      <script  type="text/javascript" charset="UTF-8" >

    var platform = new H.service.Platform({
        """
    ostr += 'app_id: "' + appId + '",\n'
    ostr += 'app_code: "' + appCode + '",\n'
      
    ostr += """
      useCIT: true,
      useHTTPS: true
    });

    var defaultLayers = platform.createDefaultLayers();

    var map = new H.Map(document.getElementById('map'), defaultLayers.normal.map);
    var behavior = new H.mapevents.Behavior(new H.mapevents.MapEvents(map));

    // Create the default UI components
    var ui = H.ui.UI.createDefault(map, defaultLayers);

    // Now use the map as required...
    // setMapViewBounds(map);

    var distanceMeasurementTool = new H.ui.DistanceMeasurement({
      'startIcon': new H.map.Icon('[START ICON URL]'),
      'stopoverIcon': new H.map.Icon('[STOPOVER ICON URL]'),
      'endIcon': new H.map.Icon('[END ICON URL]'),
      'splitIcon': new H.map.Icon('[SPLIT ICON URL]'),
      'lineStyle': {
      'strokeColor': 'rgba(18, 65, 145, .8)',
      'lineWidth': 6
      }
    });
    ui.addControl('distancemeasurement', distanceMeasurementTool);

    function moveMapToBerlin(map){
      map.setCenter({lat:52.5159, lng:13.3777});
      map.setZoom(13);
    }
    moveMapToBerlin(map);

    var svgMarkup = '<svg style="left:-14px;top:-36px;" xmlns="http://www.w3.org/2000/svg" width="28px" height="36px" ><path d="M 19 31 C 19 32.7 16.3 34 13 34 C 9.7 34 7 32.7 7 31 C 7 29.3 9.7 28 13 28 C 16.3 28 19 29.3 19 31 Z" fill="#000" fill-opacity=".2"></path><path d="M 13 0 C 9.5 0 6.3 1.3 3.8 3.8 C 1.4 7.8 0 9.4 0 12.8 C 0 16.3 1.4 19.5 3.8 21.9 L 13 31 L 22.2 21.9 C 24.6 19.5 25.9 16.3 25.9 12.8 C 25.9 9.4 24.6 6.1 22.1 3.8 C 19.7 1.3 16.5 0 13 0 Z" fill="#fff"></path><path d="M 13 2.2 C 6 2.2 2.3 7.2 2.1 12.8 C 2.1 16.1 3.1 18.4 5.2 20.5 L 13 28.2 L 20.8 20.5 C 22.9 18.4 23.8 16.2 23.8 12.8 C 23.6 7.07 20 2.2 13 2.2 Z" fill="${COLOR}"></path><text transform="matrix( 1 0 0 1 13 18 )" x="0" y="0" fill-opacity="1" fill="#fff" text-anchor="middle" font-weight="bold" font-size="13px" font-family="arial">${TEXT}</text></svg>'

    var markergroup = new H.map.Group();
    var linkgroup = new H.map.Group();
    var arrowstyle = new H.map.ArrowStyle();
    """

    missing_links = []

    # make links from shape points
    if links is not None and len(links) > 0:
        pre_link = None
        start_coord = None
        end_coord = None
        nshapes = None
        #for shapes, linkdir, tag, prob, nss in links:
        for ilink, link in enumerate(links):
            linkid = link.linkid
            if pre_link is not None and pre_link.linkid == linkid: continue
            nshapes = len(link.shapepoints)
            if nshapes == 0:
                missing_links.append(linkid)
                continue
            stripname = 'strip_' + str(ilink)
            ostr += "var " + stripname + " = new H.geo.Strip();\n"
            for ishape, shape in enumerate(link.shapepoints):
                latlon = shape
                coordname = 'coord_' + str(ilink) + '_' + str(ishape)
                ostr += 'var ' + coordname + ' = new H.geo.Point(' + latlon.slat + ',' + latlon.slon + ');\n'
                ostr += coordname + '.$label = "linkid: ' + linkid + ', location: ' + latlon.slat + ',' + latlon.slon + '";\n'
                ostr += stripname + ".pushPoint(" + coordname + ");\n"
            plname = 'pl_' + str(ilink)

            linecolor = link.color
            mcolor = link.color
            linewidth = link.width
            ostr += "var " + plname + " = new H.map.Polyline(" + stripname + ", { style: { lineWidth: '" + linewidth + "', strokeColor: '" + linecolor + "'}});\n"
            ostr += plname + '.$label = "linkid: ' + linkid + ', shapepoints: ' + link.shapepoints_str() + '";\n'
            ostr += "linkgroup.addObject(" + plname + ");\n"

            circlename = "circ_" + str(ilink) + "_0"
            coordname = 'coord_' + str(ilink) + '_0'
            ostr += 'var ' + circlename + ' = new H.map.Circle(' + coordname + ',0.5,{style: {strokeColor:"' + mcolor + '",fillColor:"' + mcolor + '"}});\n'
            ostr += circlename + ".$label = " + coordname + ".$label;\n"
            ostr += "markergroup.addObject(" + circlename + ");\n"

            circlename = "circ_" + str(ilink) + "_1"
            coordname = 'coord_' + str(ilink) + '_' + str(nshapes - 1)
            ostr += 'var ' + circlename + ' = new H.map.Circle(' + coordname + ',0.5,{style: {strokeColor:"' + mcolor + '",fillColor:"' + mcolor + '"}});\n'
            ostr += circlename + ".$label = " + coordname + ".$label;\n"
            ostr += "markergroup.addObject(" + circlename + ");\n"

            if ilink == 0:
                start_coord = 'coord_0_0'
                end_coord = 'coord_0_' + str(nshapes - 1)
            else:
                npre_shapes = len(pre_link.shapepoints)
                if ilink == 1:
                    if pre_link.shapepoints[0] == link.shapepoints[0]: start_coord = 'coord_0_' + str(npre_shapes - 1)
                    if pre_link.shapepoints[0] == link.shapepoints[nshapes-1]: start_coord = 'coord_0_' + str(npre_shapes - 1)
                end_coord = 'coord_' + str(ilink) + '_' + str(nshapes - 1)
                if pre_link.shapepoints[0] == link.shapepoints[nshapes-1]: end_coord = 'coord_' + str(ilink) + '_0'
                if pre_link.shapepoints[npre_shapes-1] == link.shapepoints[nshapes-1]: end_coord = 'coord_' + str(ilink) + '_0'

            pre_link = link
    ostr += "\n"

    # make lat/lon markers from rawlocs
    if points is not None and len(points) > 0:
        for ip, p in enumerate(points):
            ostr += 'var point_coord' + str(ip) + ' = new H.geo.Point(' + p.slat + ',' + p.slon + ');\n'
            iconname = 'rawIcon_' + str(ip)
            ostr += iconname + ' = new H.map.Icon(svgMarkup.replace("${COLOR}", "' + p.color + '").replace("${TEXT}", "' + p.text + '"));'
            ostr += 'var point_sm' + str(ip) + ' = new H.map.Marker(point_coord' + str(
                ip) + ',{icon:' + iconname + '});\n'
            ostr += 'point_sm' + str(ip) + '.setData(' + str(ip) + ');\n'
            ostr += 'point_sm' + str(ip) + '.$label = "' + p.slat + ',' + p.slon + '";\n'
            ostr += 'markergroup.addObject(point_sm' + str(ip) + ');\n'

    # make bubble info box when objects are clicked
    ostr += """

    linkgroup.addEventListener("tap", function(evt) {
      var item = evt.target;
      if (item instanceof H.map.Polyline) {
        var start_point = item.getStrip().extractPoint(0);
        ui.addBubble(new H.ui.InfoBubble(start_point,{content: item.$label}));
      };
    },
    false);
    map.addObject(linkgroup);

    markergroup.addEventListener("tap", function(evt) {
      var item = evt.target;
      if (item instanceof H.map.Marker) {
        ui.addBubble(new H.ui.InfoBubble(item.getPosition(),{content: item.$label}));
      };
      if (item instanceof H.map.Circle) {
        ui.addBubble(new H.ui.InfoBubble(item.getCenter(),{content: item.$label}));
      };
    },
    false);
    map.addObject(markergroup);
    if (markergroup.getBounds() != null) {
      map.setViewBounds(markergroup.getBounds());
    }

    """

    # draw map and zoom according to the bounding box

    # if filled: ostr += 'map.setViewBounds(circlegroup.getBounds());\n'

    ostr += '</script>\n'
    ostr += '</body>\n'

    #ostr += '<font size="2"><b>Legend</b> : <font color="blue">Links are in blue. The blue marker with "S" represents the starting point and the black marker with "E" the last point. Blue circles on the link indicate nodes of links. Arrows on the link indicate where the reference nodes are, not direction of travel. </font>\n'
    #ostr += '<font color="red">Orange markers represent locations provided in LatLons fields with the preserved order. </font>'
    #ostr += 'Click links or markers to see more information.</font>\n'

    if len(missing_links) > 0:
        ostr += '<br>\n'
        ostr += 'number of links not in database : ' + str(len(missing_links)) + '\n'
        prn_links = []
        for i, l in enumerate(missing_links):
            if i != 0 and i % 5 == 0:
                ostr += '<br>'
                ostr += ','.join(prn_links)
                prn_links = []
            prn_links.append(l)
        ostr += '<br>'
        ostr += ','.join(prn_links)
        ostr += '<br>\n'

    ostr += "</html>"

    return ostr


In [2]:
# --------------------------------
#     How to use the function 
# --------------------------------
#
#     a point object consists of (lat, lon, color, text)
#     Note: color and text can be omitted.
#     Expected Inputs:
#         raw_points: (point1, point2, ...)
#         raw_links: ((linkid,(point1, point2, ...),color,text,width)
#         Note: color, text, width for a link object can be omitted
#         map_width (optional, default=1200px): width of map as px(pixel) or %(relative ratio to the browser)
#         map_height (optional, default=600px): height of map as px(pixel). It has to be in px.
#     gethtml function transforms raw_points, raw_links to lists of Point and Link objects
#     and return html document string that can be saved to a file

appId = "70c2JISMJu0xZokMwam6"     # replace this with yours
appCode = "DU04oKdD8cRE7pqd_Xknog" # replace this with yours
raw_p1 = (52.799751,12.70337,"red","1")
raw_p2 = (52.799526,12.703685, "red", "2")
raw_p3 = (52.799324,12.704025, "red", "3")
points = [raw_p1, raw_p2, raw_p3]

# draw only points
html = gethtml(points, [], appId, appCode, map_width="1200px", map_height="600px")

fout = open("custom_markers.html","w")
fout.write(html)
fout.close()

In [4]:
mm_p1 = (52.799780, 12.703421, "blue", "1")
mm_p2 = (52.799553, 12.703742, "blue", "2")
mm_p3 = (52.799335, 12.704046, "blue", "3")
points = [raw_p1, raw_p2, raw_p3, mm_p1, mm_p2, mm_p3]
links = [
    ["link1",(raw_p1, mm_p1), "black", "1-1", "2"],
    ["link2",(raw_p2, mm_p2), "black", "2-2", "2"],
    ["link3",(raw_p3, mm_p3), "black", "3-3", "2"],
]

# draw points and links together
html = gethtml(points, links, appId, appCode, map_width="1200px", map_height="600px")

fout = open("custom_markers2.html","w")
fout.write(html)
fout.close()